importing libraries

In [27]:
import matplotlib.pylab as plt
import numpy as np
from pandas import read_csv,DataFrame
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

loading the dataset

In [28]:
data=read_csv("../data/california_housing.csv")

after using isna it turns out that there is no missing data.

In [29]:
isMissing=data.isna().any().any()
print(isMissing)

False


1. standardizing the features
2. creating a dataframe from numpy matrix

In [30]:
x=data.drop("MedianHouseValue",axis=1)
y=data["MedianHouseValue"]
scaler=StandardScaler()
scaledXNP=scaler.fit_transform(x)
scaledXDF=DataFrame(scaledXNP,columns=x.columns)

for the first part we only going to focus on using one feature which is `MedInc`

In [31]:
x0=scaledXDF["MedInc"]
x0Train,x0Test,y0Train,y0Test=train_test_split(x0,y,random_state=42,test_size=0.3)

training a linear regression model using just `MedInc` feature and testing it performance with mse.

**Note**: the fit function expect a matrix so we need to reshape the x0Train 

In [32]:
model0=LinearRegression()
model0.fit(x0Train.values.reshape(-1,1),y0Train)
pred0=model0.predict(X=x0Test.values.reshape(-1,1))
error0=mean_squared_error(y_true=y0Test,y_pred=pred0)
print(error0,np.mean(y0Test))

0.6917979868048499 2.066968142764858


1. The error1 is more than a quarter of the y0Test mean which is quite substantiel
2. We going to include the remaining features.

In [37]:
x1Train,x1Test,y1Train,y1Test=train_test_split(scaledXDF,y,random_state=42,test_size=0.3)
model1=LinearRegression()
model1.fit(x1Train,y1Train)
pred1=model1.predict(X=x1Test)
error1=mean_squared_error(y_true=y1Test,y_pred=pred1)
print(error1,np.mean(y1Test))

0.5305677824766754 2.066968142764858


Even though we have used all the features that did not result in a big improvment from result of the last model just a difference of `1.6`

What we can do now is try transforming our features to polynomial in order to reduce the error.To be clear here applying polynomial transform with a degree=2 to all features gives the best results.

In [38]:
model2=make_pipeline(PolynomialFeatures(2),LinearRegression())
model2.fit(x1Train,y1Train)
pred2=model2.predict(X=x1Test)
error2=mean_squared_error(y_true=y1Test,y_pred=pred2)
print(error2,np.mean(y1Test))

0.45497233748568755 2.066968142764858


As we can see the error has decreased not with a big margin.

**Note**: even though after plotting multiple graphs for featureX/target it seems some of them has no effect on the `medianHouseValue` removing them however will actually have negative impact on the error 

The final error will be `0.504` if we drop AveOccup.